# 06_Downloading_PostgreSQL_CSV

In [1]:
import pandas as pd
import sys
import numpy as np
import os
import requests
import datetime
import base64
import json
import re
from collections import Counter
import psycopg2
import tqdm
from bs4 import BeautifulSoup
from importlib import reload
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# We have created a list of functions in db_utils.py
import db_utils
#reload(db_utils)

In [3]:
conn = psycopg2.connect("dbname=spotify")
cursor= conn.cursor()

In [4]:
out = "../data/psql_out/"

# Nodes

## 1. Master_Track

In [4]:
# Extrack all the artists
df_track = db_utils.select_table(cursor, table="master_track", 
                                 sel_cols=["track_id", "track_name", "peak_date", "streams"])

# Replace strings that may be conflictive
df_track["track_name"] = df_track["track_name"].str.replace(";", "")

#### Export to csv

In [18]:
df_track.to_csv(out + "master_track.csv", header=True, sep=";", index=False)

## 2. Master_Artist

In [19]:
# Extrack all the artists
df_artist = db_utils.select_table(cursor, table="master_artist", sel_cols=["artist_id", "artist_name", "is_main"])

# Extract the popularity for the related artist
df_rel_artist = db_utils.select_table(cursor, table="rel_artist_artist", sel_cols=["query","rel_art", "popularity"])
df_rel_artist.drop_duplicates(inplace=True)

# Select only the two columns of artist - popularity
df_popularity = df_rel_artist[["rel_art", "popularity"]].copy()
df_popularity.drop_duplicates(inplace=True)

# Convert column names
df_popularity.columns = ["artist_id", "popularity"]

# Do a left join to bring popularity to the Master Artist
df_merged = pd.merge(df_artist, df_popularity, how = 'left', on = "artist_id")

Now for the artist with NaN as popularity we will look for the relationships with other artists

In [35]:
# Artists without popularity score
artist_no_pop = df_merged[df_merged["popularity"].isna()]

In [59]:
# Final dataframe where the inferred popularity will be also update
df_artist_final = df_merged.copy()
df_artist_final = df_artist_final.set_index("artist_id")

for i, row in tqdm.tqdm_notebook(artist_no_pop.iterrows()):
    
    artist_id = row.artist_id
    
    # Look for the similar artists' in df_rel_artist popularity
    df_similar_artist = df_rel_artist[df_rel_artist["query"] == artist_id]
    
    if df_similar_artist.shape[0] > 0:
        pop = int(np.round(df_similar_artist["popularity"].mean()))
    else:
        pop = 0
        
    # Update the final dataframe with this retrieved popularity
    df_artist_final.loc[artist_id, "popularity"] = pop    
    
# Reset the index to have a column named artist_id
df_artist_final.reset_index(inplace=True)

/Users/david/.local/share/virtualenvs/Spotify-xJiBt4R8/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [65]:
# Correct the ; in the name to avoid using a delimiter that appears on the name of the artists
df_artist_final["artist_name"] = df_artist_final["artist_name"].str.replace(";", " & ") # two artist at the same time

#### Export to csv

In [ ]:
df_artist_final.to_csv(out + "master_artist.csv", header=True, sep=";", index=False)

## 3. Master_Genre

In [75]:
# Extrack all the genres
df_genre = db_utils.select_table(cursor, table="master_genre", 
                                 sel_cols=["genre"])
df_genre.drop_duplicates(inplace=True)

In [77]:
# Avoid conflictive strings with the ; separator
df_genre["genre"] = df_genre["genre"].str.replace(";", "") 

#### Export to csv

In [78]:
df_genre.to_csv(out + "master_genre.csv", header=True, sep=";", index=False)

# Relationships

## GEN_ART

In [5]:
# Extract the popularity for the related artist
df_GEN_ART = db_utils.select_table(cursor, table="master_genre", sel_cols=["genre","artist_id"])
df_GEN_ART.drop_duplicates(inplace=True)
df_GEN_ART["genre"] = df_GEN_ART["genre"].str.replace(";", "") 

#### Export to csv

In [8]:
df_GEN_ART.to_csv(out + "rel_GEN_ART.csv", header=True, sep=";", index=False)

## ART_TR

In [11]:
# Extract the popularity for the related artist
df_ART_TR = db_utils.select_table(cursor, table="rel_artist_track", sel_cols=["artist_id","track_id"])
df_ART_TR.drop_duplicates(inplace=True)

#### Export to csv

In [13]:
df_ART_TR.to_csv(out + "rel_ART_TR.csv", header=True, sep=";", index=False)

## REL_ART

In [17]:
# Extract the popularity for the related artist
df_REL_ART = db_utils.select_table(cursor, table="rel_artist_artist", sel_cols=["query","rel_art"])
df_REL_ART.drop_duplicates(inplace=True)
df_REL_ART.columns = ["artist_id1", "artist_id2"]

#### Export to csv

In [18]:
df_REL_ART.to_csv(out + "rel_REL_ART.csv", header=True, sep=";", index=False)

# FINAL STEP

Move manually those .csv files to the folder IMPORt of the instance of the database (go to Neo4j Browser, Click on the 3 points of the database, click on Manage and see the Open Folder, in which direction it opens the folder). In our case:

/Users/david/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-ada73e8c-396c-4507-82cc-758b5f072ea4/installation-4.0.4/import